In [4]:
cd Desktop/Summer\ research/fem/

[Errno 2] No such file or directory: 'Desktop/Summer research/fem/'
/Users/wuyifan/Desktop/Summer research/fem


In [5]:
import numpy as np
import math
import igl
import meshplot as mp
import wildmeshing as wm
import polyfempy as pf

V, F = igl.read_triangle_mesh("data/mesh.obj")
#p1 = mp.plot(V, F, shading={"wireframe": True}, return_plot=True)
#p1.save("cylinder")

wm.tetrahedralize("data/mesh.obj", "out/out_mesh.mesh", mute_log=True)
solver = pf.Solver()
solver.load_mesh_from_path("out/out_mesh.mesh")

In [6]:
minn = np.min(V, axis=0)
maxx = np.max(V, axis=0)
center = (minn+maxx)/2

def sideset(p):
    if abs(p[2] - minn[2]) < 0.5:
        if p[1] > center[1]:
            return 2
        else:
            return 3
    return 1

solver.set_boundary_side_set_from_bary(sideset)

"""s: categories by pts"""
ps, ts, s = solver.get_boundary_sidesets()

e1 = ps[s[:,0] == 1, :]
e2 = ps[s[:,0] == 2, :]
e3 = ps[s[:,0] == 3, :]

col = np.zeros_like(s); col[s==2] = 2; col[s==3] = 3
plot2 = mp.plot(ps, ts)
#plot2.save("col")

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, -34.…

In [27]:
settings = pf.Settings()
problem = pf.Problem()

settings.set_pde(pf.PDEs.LinearElasticity)

settings.set_material_params("E", 20000)
settings.set_material_params("nu", 0.35)

# set the displacement value for the sideset id
problem.set_displacement(2, [0, 0, 0])
problem.set_force(3, [0, -5, 0])

settings.set_problem(problem)

solver.settings(settings)
solver.solve()

p, tri, disp = solver.get_sampled_solution()

m = np.linalg.norm(disp, axis=1)

In [24]:
plot = mp.plot(v=p+disp, f=tri, c=m, return_plot=True)
plot.add_points(e1, shading={"point_color": "yellow", "point_size": 10});
plot.add_points(e2, shading={"point_color": "red", "point_size": 10});
plot.add_points(e3, shading={"point_color": "green", "point_size": 10});

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.014812…

In [68]:
"""
Find the unique elements of an array
Returns: 
    sorted unique elements of an array
    indices of the first occurrences of the unique values in the original array
    indices to reconstruct the original array from the unique array, len = original
        inverse[i]=x <=> p[i]=p_uni[x]
            x: index in p_uni
            i: index in original array
"""
print(len(p))
p_uni, indices, inverse = np.unique(p, return_index=True, return_inverse=True, axis=0)
print(len(p_uni))

"""original tris with indices in unique array"""
t_uni = np.array([inverse[tri[:, 0]], inverse[tri[:, 1]], inverse[tri[:, 2]], inverse[tri[:, 3]]]).transpose()
d_uni = disp[indices, :]
m_uni = m[indices]

"""
Constructs the tet_tet adjacency matrix for a given tet mesh with tets T
Returns
    TT #T by #4 adjacency matrix, the element [i,j] is the id of the tet adjacent to the face j of tet i
    TTi #T by #4 adjacency matrix, the element i,j is the id of face of the tet TT(i,j) that is adjacent to tet i
"""
adj, _ = igl.tet_tet_adjacency(t_uni)

igl_faces = [[0,1,2], [0,1,3], [1,2,3], [2,0,3]]
surf = []
#print(t_uni[12,2])
for t in range(adj.shape[0]):
    for f in range(adj.shape[1]):
        face = igl_faces[f]
        """face f of tet t has no neighbour"""
        if adj[t, f] == -1:
            #print("t = ", t, ", f = ",f, ", t_uni_pt1 = ",t_uni[t, face[0]])
            """pick 3 points in t with indices in t_uni"""
            surf += [[t_uni[t, face[0]], t_uni[t, face[1]], t_uni[t, face[2]]]]
surf = np.array(surf)
print(surf.shape,p_uni.shape,t_uni.shape)

"""
Constructs isolines for a function z given on a mesh (V,F)
    *f must be triangles*
return
    isoV: #isoV by dim list of isoline vertex positions
    isoE: #isoE by 2 list of isoline edge positions
"""
iso_p, iso_l = igl.isolines(v=p_uni+d_uni, f=surf, z=m_uni, n=40)

plot = mp.plot(p_uni+d_uni, surf, m_uni, return_plot=True)

mag = np.array([0, -5, 0])

plot.add_lines(np.array([[0,0,0],[100,0,0],[100,0,0]]), np.array([[100,0,0],[95,5,0],[95,-5,0]]), shading={"line_color": "blue"})
plot.add_lines(np.array([[0,0,0],[0,100,0],[0,100,0]]), np.array([[0,100,0],[-5,95,0],[5,95,0]]), shading={"line_color": "red"})
plot.add_lines(np.array([[0,0,0],[0,0,100],[0,0,100]]), np.array([[0,0,100],[0,5,95],[0,-5,95]]), shading={"line_color": "yellow"})

dir1 = -mag/5
dir2 = -mag/5
dir1 += [-1,0,0]
dir2 += [1,0,0]

e31 = np.vstack((np.vstack((e3[0:20],e3[0:20])),e3[0:20]))
e32 = np.vstack((np.vstack((e3[0:20]-mag,e3[0:20]+dir1)),e3[0:20]+dir2))
plot.add_lines(e31, e32, shading={"line_color": "black"})
e33 = np.vstack((np.vstack((e3[30:50],e3[30:50])),e3[30:50]))
e34 = np.vstack((np.vstack((e3[30:50]-mag,e3[30:50]+dir1)),e3[30:50]+dir2))
plot.add_lines(e33, e34, shading={"line_color": "black"})

#plot.add_lines(e3, e3+mag, shading={"line_color": "black"})
#plot.add_lines(e3+mag, e3+mag+dir1, shading={"line_color": "black"})
#plot.add_lines(e3+mag, e3+mag+dir2, shading={"line_color": "black"})
#plot.add_edges(iso_p, iso_l, shading={"line_color": "white"})
plot.add_points(e2, shading={"point_color": "white", "point_size": 7})

#plot.save("isoline")

141270
23946
(18992, 3) (23946, 3) (113016, 4)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.148120…

6